In [9]:
import numpy as np 
from sklearn.model_selection import train_test_split 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf


mainData = pd.read_csv('./data_labels_mainData.csv')
extraData = pd.read_csv('./data_labels_extraData.csv')
frames = [mainData, extraData]
isCancerousData = pd.concat(frames)

In [2]:
#drop IDs
isCancerousData.drop(['InstanceID', 'patientID', 'cellType'], axis = 1, inplace =True)
isCancerousData['ImgPath'] = './patch_images/' + isCancerousData['ImageName']

In [3]:
CLASSES = 2

train_data, test_data = train_test_split(isCancerousData, test_size=0.2, random_state=42) 

train_data, val_data = train_test_split(isCancerousData, test_size=0.25, random_state=42) 

print("Train data : {}, Val Data: {}, Test Data: {}".format(train_data.shape[0], val_data.shape[0], test_data.shape[0])) 

target = train_data['isCancerous']

Train data : 15210, Val Data: 5070, Test Data: 4056


In [4]:
train_data['isCancerous'] = train_data['isCancerous'].astype('str')
val_data['isCancerous'] = val_data['isCancerous'].astype('str')

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last',
                                   rotation_range=15, width_shift_range=0.2,
                                   height_shift_range=0.2, brightness_range=[0.5,1.5])
val_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')

batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_data,
        directory='./',
        x_col="ImgPath",
        y_col="isCancerous",
        target_size=(27, 27),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = val_datagen.flow_from_dataframe(
        dataframe=val_data,
        directory='./',
        x_col="ImgPath",
        y_col="isCancerous",
        target_size=(27, 27),
        batch_size=batch_size,
        class_mode='binary')

Found 15210 validated image filenames belonging to 2 classes.
Found 5070 validated image filenames belonging to 2 classes.


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [5]:
import matplotlib.pyplot as plt
def plot_learning_curve(train_loss, val_loss, train_metric, val_metric, metric_name='Accuracy'):
    plt.figure(figsize=(10,5))
    
    plt.subplot(1,2,1)
    plt.plot(train_loss, 'r--')
    plt.plot(val_loss, 'b--')
    plt.xlabel("epochs")
    plt.ylabel("Loss")
    plt.legend(['train', 'val'], loc='upper left')

    plt.subplot(1,2,2)
    plt.plot(train_metric, 'r--')
    plt.plot(val_metric, 'b--')
    plt.xlabel("epochs")
    plt.ylabel(metric_name)
    plt.legend(['train', 'val'], loc='upper left')

    plt.show()

In [20]:
import numpy as np

# (3) Create a sequential model
model = tf.keras.Sequential([
    # 1st Convolutional Layer
    tf.keras.layers.Conv2D(filters=96, input_shape=(27,27,3), kernel_size=(11,11),\
                           strides=(4,4), padding='valid', activation='relu'),
    # Pooling 
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'),
    # Batch Normalisation before passing it to the next layer
    tf.keras.layers.BatchNormalization(),

    # 2nd Convolutional Layer
    tf.keras.layers.Conv2D(filters=256, kernel_size=(11,11), strides=(1,1),\
                       padding='valid', activation='relu'),
    # Pooling
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'),
    tf.keras.layers.Activation('relu'),
    
    # Batch Normalisation
    tf.keras.layers.BatchNormalization(),

    # 3rd Convolutional Layer
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid',\
    activation='relu'),
    tf.keras.layers.BatchNormalization(),

    # 4th Convolutional Layer
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid',\
    activation='relu'),
    # Batch Normalisation
    tf.keras.layers.BatchNormalization(),
    
    # 5th Convolutional Layer
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid',\
             activation='relu'),
    # Pooling
    tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'),
    # Batch Normalisation
    tf.keras.layers.BatchNormalization(),

    # Passing it to a dense layer
    tf.keras.layers.Flatten(),
    # 1st Dense Layer
    tf.keras.layers.Dense(4096, input_shape=(27*27*3,)),
    tf.keras.layers.Activation('relu'),
    # Add Dropout to prevent overfitting
    tf.keras.layers.Dropout(0.4),
    # Batch Normalisation
    tf.keras.layers.BatchNormalization(),

    # 2nd Dense Layer
    tf.keras.layers.Dense(4096),
    tf.keras.layers.Activation('relu'),
    # Add Dropout
    tf.keras.layers.Dropout(0.4),
    # Batch Normalisation
    tf.keras.layers.BatchNormalization(),

    # 3rd Dense Layer
    tf.keras.layers.Dense(1000),
    tf.keras.layers.Activation('relu'),
    # Add Dropout
    tf.keras.layers.Dropout(0.4),
    # Batch Normalisation
    tf.keras.layers.BatchNormalization(),

    # Output Layer
    tf.keras.layers.Dense(17),
    tf.keras.layers.Activation('softmax'),
    
])

ValueError: Negative dimension size caused by subtracting 11 from 2 for '{{node conv2d_35/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, conv2d_35/Conv2D/ReadVariableOp)' with input shapes: [?,2,2,96], [11,11,96,256].

In [ ]:
# (4) Compile 
model.compile(loss='binary_crossentropy', optimizer='adam',\
 metrics=['accuracy'])

# (5) Train
history = model.fit(train_generator, validation_data, batch_size=64, epochs=1, verbose=1, \
validation_split=0.2, shuffle=True)

plot_learning_curve(history.history['loss'], history.history['val_loss'], 
                    history.history['binary_accuracy'], vgg_model_history.history['val_binary_accuracy'], 
                    metric_name='Accuracy')